Basically doing the same thing, but with CPLEX

In [124]:
time_brackets = [8, 12, 16, 20]

In [125]:
import json

def get_transport_matrix():
    BASE_PATH = "../data/routeData/"
    daytimes = [("morning", 8), ("midday", 12), ("evening", 16), ("night", 20)]
    transport_matrix = {}

    for time_in_day, hour in daytimes:
        filepath = BASE_PATH + f"route_matrix_{time_in_day}.json"

        with open(filepath, 'r') as f:
            route_matrix = json.load(f)

        for route in route_matrix["routes"]:
            this_route = route_matrix["routes"][route]
            origin = this_route["origin_name"]
            destination = this_route["destination_name"]
            transport_matrix[(origin, destination, hour)] = {
                "transit": {
                    "duration": this_route["transit"]["duration_minutes"],
                    "price": this_route["transit"]["fare_sgd"],
                },
                "drive": {
                    "duration": this_route["drive"]["duration_minutes"],
                    "price": this_route["drive"]["fare_sgd"],
                }
            }
    
    return transport_matrix


def get_all_locations():
    BASE_PATH = "../data/routeData/"

    with open(BASE_PATH + "route_matrix_morning.json", 'r') as f:
        route_matrix = json.load(f)
    
    locations = [route_matrix["locations"][location_id] for location_id in route_matrix["locations"]]
    return locations


def get_transport_hour(transport_time):
    # because the transport_matrix is only bracketed to 4 groups, we find the earliest it happens
    transport_hour = transport_time // 60

    for bracket in reversed(time_brackets):
        if transport_hour >= bracket:
            return bracket

    return transport_brackets[-1] # from 8 PM to 8 AM next day


In [126]:
import numpy as np

In [127]:
all_locations = get_all_locations()

# for all locations, get satisfaction and rating
for loc in all_locations:
    if loc["type"] == "hawker":
        loc["rating"] = np.random.randint(0, 5)
        loc["avg_food_price"] = np.random.randint(5, 15)
        loc["duration"] = 60 # just standardize 60 mins
    elif loc["type"] == "attraction":
        loc["satisfaction"] = np.random.randint(0, 10)
        loc["entrance_fee"] = np.random.randint(5, 100)
        loc["duration"] = np.random.randint(30, 90)

dummy_hotel = {
    "type": "hotel",
    "name": "DUMMY HOTEL",
    "lat": 1.2852044,
    "lng": 103.8610313,
}

In [128]:
transport_matrix = get_transport_matrix()
# add dummy hotel to transport_matrix
for loc in all_locations:
    for time_ in time_brackets:
        transport_matrix[(dummy_hotel["name"], loc["name"], time_)] = {
            "transit": {
                "duration": 50,
                "price": 1.93,
            },
            "drive": {
                "duration": 20,
                "price": 10.1,
            }
        }
        transport_matrix[(loc["name"], dummy_hotel["name"], time_)] = {
            "transit": {
                "duration": 50,
                "price": 1.93,
            },
            "drive": {
                "duration": 20,
                "price": 10.1,
            }
        }

locations = [dummy_hotel] + all_locations

In [129]:
import random
random.seed(42)

hotels = [loc for loc in locations if loc["type"] == "hotel"]
attractions = [loc for loc in locations if loc["type"] == "attraction"]
hawkers = [loc for loc in locations if loc["type"] == "hawker"]

selected_hotel = random.sample(hotels, 1)
selected_attractions = random.sample(attractions, 5)
selected_hawkers = random.sample(hawkers, 2)

selected_locations = selected_hotel + selected_attractions + selected_hawkers

In [130]:
selected_locations

[{'type': 'hotel',
  'name': 'DUMMY HOTEL',
  'lat': 1.2852044,
  'lng': 103.8610313},
 {'id': 'A4',
  'name': 'ArtScience Museum',
  'type': 'attraction',
  'lat': 1.2862738,
  'lng': 103.8592663,
  'satisfaction': 0,
  'entrance_fee': 46,
  'duration': 55},
 {'id': 'A7',
  'name': 'National Museum of Singapore',
  'type': 'attraction',
  'lat': 1.296613,
  'lng': 103.8485091,
  'satisfaction': 0,
  'entrance_fee': 66,
  'duration': 35},
 {'id': 'A13',
  'name': 'Esplanade',
  'type': 'attraction',
  'lat': 1.2897934,
  'lng': 103.8558166,
  'satisfaction': 8,
  'entrance_fee': 29,
  'duration': 84},
 {'id': 'A11',
  'name': 'Clarke Quay',
  'type': 'attraction',
  'lat': 1.2906024,
  'lng': 103.8464742,
  'satisfaction': 7,
  'entrance_fee': 46,
  'duration': 71},
 {'id': 'A12',
  'name': 'Orchard Road',
  'type': 'attraction',
  'lat': 1.3048205,
  'lng': 103.8321984,
  'satisfaction': 4,
  'entrance_fee': 72,
  'duration': 51},
 {'id': 'H3',
  'name': 'Hill Street Tai Hwa Pork Nood

In [187]:
from docplex.mp.model import Model

In [188]:
mdl = Model("MITB-AI")

In [189]:
NUM_DAYS = 3
HOTEL_COST = 50
START_TIME = 9 * 60 # everyday starts from 9 AM
HARD_LIMIT_END_TIME = 22 * 60 # everyday MUST return back to hotel by 10 PM

# Define lunch and dinner time windows (in minutes since start of day at 9 AM)
LUNCH_START = 11 * 60  # 11 AM (9 AM + 2 hours)
LUNCH_END = 15 * 60    # 3 PM (9 AM + 6 hours)
DINNER_START = 17 * 60  # 5 PM (9 AM + 8 hours)
DINNER_END = 21 * 60   # 9 PM (9 AM + 12 hours)

# hard limit money spent
budget = 1000
# list of destinations
locations = locations
num_locations = len(locations)
num_attractions = len([loc for loc in locations if loc["type"] == "attraction"])
num_hawkers = len([loc for loc in locations if loc["type"] == "hawker"])
num_hotels = 1 # always 1

# transportation option prices and durations. The size MUST BE num_dest * num_dest, for 24 hours
transport_types = ["transit", "drive"]
num_transport_types = len(transport_types)

In [190]:
x_var = {
    (day, transport_type, source["name"], dest["name"]): mdl.binary_var(f"Day_{day}_GoFrom_{source['name']}_To_{dest['name']}_using_{transport_type}")
    for day in range(NUM_DAYS)
    for transport_type in transport_types
    for source in locations
    for dest in locations
    if source != dest
}

u_var = {
    (day, location["name"]): mdl.integer_var(0, HARD_LIMIT_END_TIME, name=f"Day_{day}_FinishAttraction_{location['name']}_time")
    for day in range(NUM_DAYS)
    for location in locations
}

bracket_var = {
    (day, location["name"], time_bracket): mdl.binary_var(f"Day_{day}_FinishAttraction_{location['name']}_time_bracket_{time_bracket}")
    for day in range(NUM_DAYS)
    for location in locations
    for time_bracket in time_brackets
}

last_visit_var = {
    (day, location["name"]): mdl.binary_var(name=f"Day_{day}_LastVisitIs_{location['name']}")
    for day in range(NUM_DAYS)
    for location in locations
}

lunch_hawker_visit_var = {
    (day, location["name"]): mdl.binary_var(name=f"Day_{day}_VisitHawker_{location['name']}_inLunchTime")
    for day in range(NUM_DAYS)
    for location in locations
    if location["type"] == "hawker"
}

dinner_hawker_visit_var = {
    (day, location["name"]): mdl.binary_var(name=f"Day_{day}_VisitHawker_{location['name']}_inDinnerTime")
    for day in range(NUM_DAYS)
    for location in locations
    if location["type"] == "hawker"
}

cost_var = mdl.continuous_var(lb=0, name="total_cost")
travel_time_var = mdl.continuous_var(lb=0, name="total_travel_time")
satisfaction_var = mdl.continuous_var(lb=0, name="total_satisfaction")

In [191]:
# every day, for every location, the time bracket the guy goes in, must be only 1, cannot go to multiple
for day in range(NUM_DAYS):
    for source in locations:
        mdl.add_constraint(
            sum(bracket_var[(day, source["name"], time_bracket)] for time_bracket in time_brackets) == 1
        )

# link bracket_var and u_var
LARGE_M = 10e6  # Large enough big-M constant
for day in range(NUM_DAYS):
    for source in locations:
        for time_bracket in time_brackets:
            mdl.add_constraint( # ensure that if this time bracket is chosen, then u_var must be bigger than the next bracket
                time_bracket * bracket_var[(day, source["name"], time_bracket)] <= u_var[(day, source["name"])].divide(60)
            )
            # if this time bracket is chosen, then it must be between this bracket and the next bracket
            mdl.add_constraint(
                u_var[(day, source["name"])].divide(60) <= (time_bracket + 4) * bracket_var[(day, source["name"], time_bracket)] +
                LARGE_M * (1 - bracket_var[(day, source["name"], time_bracket)])
            )

# make sure there's only 1 last visit every day
for day in range(NUM_DAYS):
    mdl.add_constraint(
        sum([last_visit_var[(day, loc["name"])] for loc in locations[1:]]) == 1
    )
    # link last visit with u_var
    for loc in locations:
        for other_loc in locations:
            mdl.add_constraint(
                u_var[(day, loc["name"])] <= u_var[(day, other_loc["name"])] +
                LARGE_M * (1 - last_visit_var[(day, other_loc["name"])])
            )
    # for everyday, last visit cannot be hotel
    mdl.add_constraint(last_visit_var[(day, locations[0]["name"])] == 0)

# for every attraction, must be a source & destination at most once
for loc in locations:
    if loc["type"] == "attraction":
        mdl.add_constraint(
            sum([
                x_var[(day, transport_type, loc["name"], dest["name"])]
                for day in range(NUM_DAYS)
                for transport_type in transport_types
                for dest in locations
                if loc != dest
            ]) <= 1
        )
        mdl.add_constraint(
            sum([
                x_var[(day, transport_type, source["name"], loc["name"])]
                for day in range(NUM_DAYS)
                for transport_type in transport_types
                for source in locations
                if loc != source
            ]) <= 1
        )

# for hawkers, every day must be a source & destination at most once
for day in range(NUM_DAYS):
    for loc in locations:
        if loc["type"] == "hawker":
            mdl.add_constraint(
                sum([
                    x_var[(day, transport_type, source["name"], loc["name"])]
                    for transport_type in transport_types
                    for source in locations
                    if source != loc
                ]) <= 1
            )
            mdl.add_constraint(
                sum([
                    x_var[(day, transport_type, loc["name"], dest["name"])]
                    for transport_type in transport_types
                    for dest in locations
                    if dest != loc
                ]) <= 1
            )

        # if attraction is a source, it must also be a destination
        mdl.add_constraint(
            sum([
                x_var[(day, transport_type, source["name"], loc["name"])]
                for transport_type in transport_types
                for source in locations
                if source != loc
            ]) == sum([
                x_var[(day, transport_type, loc["name"], dest["name"])]
                for transport_type in transport_types
                for dest in locations
                if dest != loc
            ])
        )

# link lunch & dinner hawker visit to u_var
for day in range(NUM_DAYS):
    for loc in locations:
        if loc["type"] != "hawker": continue

        mdl.add_constraint(
            LUNCH_START * lunch_hawker_visit_var[(day, loc["name"])] <= u_var[(day, loc["name"])]
        )
        mdl.add_constraint(
            u_var[(day, loc["name"])] <= LUNCH_END + LARGE_M * (1 - lunch_hawker_visit_var[(day, loc["name"])])
        )

        mdl.add_constraint(
            DINNER_START * dinner_hawker_visit_var[(day, loc["name"])] <= u_var[(day, loc["name"])]
        )
        mdl.add_constraint(
            u_var[(day, loc["name"])] <= DINNER_END + LARGE_M * (1 - dinner_hawker_visit_var[(day, loc["name"])])
        )

for day in range(NUM_DAYS):
    # everyday, hotel must be starting point
    for loc in locations:
        if loc["name"] != locations[0]["name"]:
            mdl.add_constraint(
                u_var[(day, locations[0]["name"])] <= u_var[(day, loc["name"])]
            )

    # hotel starting must be at START_TIME earliest
    mdl.add_constraint(
        u_var[(day, locations[0]["name"])] >= START_TIME
    )

    for transport_type in transport_types:
        for source in locations:
            for dest in locations[1:]: # NOTE here that hotel (index 0) isn't included in destination. It will be computed later
                if source == dest: continue
                # every day, if the route is chosen, the time of finishing in this place is this
                for time_bracket in time_brackets:
                    transport_value = transport_matrix[(source["name"], dest["name"], time_bracket)][transport_type]
                    mdl.add_constraint(
                        u_var[(day, dest["name"])] >= u_var[(day, source["name"])] +
                        transport_value["duration"] + dest["duration"] -
                        LARGE_M * (1 - x_var[(day, transport_type, source["name"], dest["name"])]) -
                        LARGE_M * (1 - bracket_var[(day, source["name"], time_bracket)])
                    )

    # make sure to go back to hotel
    for loc in locations[1:]:
        mdl.add_constraint(
            sum([
                x_var[(day, transport_type, loc["name"], locations[0]["name"])]
                for transport_type in transport_types
            ]) == last_visit_var[(day, loc["name"])]
        )

    for loc in locations:
        mdl.add_constraint(
            sum([
                x_var[(day, transport_type, source["name"], loc["name"])]
                for transport_type in transport_types
                for source in locations
                if source != loc
            ]) == sum([
                x_var[(day, transport_type, loc["name"], dest["name"])]
                for transport_type in transport_types
                for dest in locations
                if dest != loc
            ])
        )

    # everyday, must go to hawkers at least twice (lunch & dinner)
    mdl.add_constraint(sum([
        x_var[(day, transport_type, source["name"], dest["name"])]
        for transport_type in transport_types
        for source in locations
        for dest in locations
        if source != dest and dest["type"] == "hawker"
    ]) >= 2)

    # every day, have lunch exactly once
    mdl.add_constraint(
        sum(lunch_hawker_visit_var[(day, hawker["name"])] for hawker in locations if hawker["type"] == "hawker") == 1
    )

    # every day, have dinner exactly once
    mdl.add_constraint(
        sum(dinner_hawker_visit_var[(day, hawker["name"])] for hawker in locations if hawker["type"] == "hawker") == 1
    )

    for source in locations:
        for dest in locations:
            if source == dest: continue
            # for every day, if public transportation is chosen, taxi can't be chosen, and vice versa
            mdl.add_constraint(
                sum([
                    x_var[(day, transport_type, source["name"], dest["name"])]
                    for transport_type in transport_types
                ]) <= 1
            )

mdl.add_constraint(cost_var ==
    NUM_DAYS * HOTEL_COST + 
    sum(
        x_var[(day, transport_type, source["name"], dest["name"])] * (
            transport_matrix[(source["name"], dest["name"], time_bracket)][transport_type]["price"] 
            + (dest["entrance_fee"] if dest["type"] == "attraction" else 0)
            + (dest["avg_food_price"] if dest["type"] == "hawker" else 0)
        )
        for day in range(NUM_DAYS)
        for transport_type in transport_types
        for source in locations
        for dest in locations
        for time_bracket in time_brackets
        if source["name"] != dest["name"]
    )
)
mdl.add_constraint(travel_time_var == 
    sum(
        x_var[(day, transport_type, source["name"], dest["name"])] * 
        transport_matrix[(source["name"], dest["name"], time_bracket)][transport_type]["duration"]
        for day in range(NUM_DAYS)
        for transport_type in transport_types
        for source in locations
        for dest in locations
        for time_bracket in time_brackets
        if source["name"] != dest["name"]
    )
)
mdl.add_constraint(satisfaction_var == 
    sum(
        x_var[(day, transport_type, source["name"], dest["name"])] * (
            dest["satisfaction"] if dest["type"] == "attraction" else 0
        )
        for day in range(NUM_DAYS)
        for transport_type in transport_types
        for source in locations
        for dest in locations
        if source["name"] != dest["name"]
    ) + sum(
        x_var[(day, transport_type, source["name"], dest["name"])] * (
            dest["rating"] if dest["type"] == "hawker" else 0
        )
        for day in range(NUM_DAYS)
        for transport_type in transport_types
        for source in locations
        for dest in locations
        if source["name"] != dest["name"]
    )
)
# finally, make sure everything is within budget
mdl.add_constraint(cost_var <= budget)

# you must visit 2-6 places per day
min_visits = NUM_DAYS * 2
max_visits = NUM_DAYS * 6
mdl.add_constraint(
    sum([
        x_var[(day, transport_type, source["name"], dest["name"])]
        for day in range(NUM_DAYS)
        for transport_type in transport_types
        for source in locations
        for dest in locations
        if source != dest
    ]) <= max_visits
)
mdl.add_constraint(
    sum([
        x_var[(day, transport_type, source["name"], dest["name"])]
        for day in range(NUM_DAYS)
        for transport_type in transport_types
        for source in locations
        for dest in locations
        if source != dest
    ]) >= min_visits
)

# <ADD CONSTRAINTS HERE>
constraints = [
    """mdl.add_constraint(x_var[(day, transport_type, source["name"], dest["name"])] == 1)"""
]

In [192]:
mdl.minimize(cost_var + travel_time_var * 0.1 - satisfaction_var * 80)

In [193]:
solution = mdl.solve(log_output=True)
print(solution)

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 2964 rows and 5 columns.
MIP Presolve modified 84800 coefficients.
Reduced MIP has 22208 rows, 5464 columns, and 119683 nonzeros.
Reduced MIP has 5376 binaries, 87 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.09 sec. (99.75 ticks)
Probing fixed 6 vars, tightened 0 bounds.
Probing time = 0.04 sec. (30.97 ticks)
Cover probing fixed 0 vars, tightened 12 bounds.
Tried aggregator 2 times.
Detecting symmetries...
MIP Presolve eliminated 600 rows and 9 columns.
Aggregator did 6 substitutions.
Reduced MIP has 21602 rows, 5449 columns, and 117121 nonzeros.
Reduced MIP has 5364 binaries, 84 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.06 sec. (97.76 ticks)
Probing time = 0.01 sec. (6.89 ticks)
Cover probing fixed 0 vars, tightened 3 bounds.
Clique table members: 59398.
Tightened 3 constraints.
MIP emphasis: balance optimality 

In [200]:
for day in range(NUM_DAYS):
    print(f"Day {day}:")
    
    # Find starting location (hotel) and its start time
    start_location = locations[0]["name"]
    start_time = round(u_var[(day, start_location)].solution_value)
    print(f"- Start at {start_location} by {int(start_time // 60):02d}:{int(start_time % 60):02d}")

    current_location = start_location
    while True:
        next_location = None
        chosen_transport = None
        travel_time = None
        price = None
        entrance_fee = None
        duration = None
        food_price = None

        # Find the next location and transport used
        for transport_type in transport_types:
            for dest in locations:
                if dest["name"] == current_location:
                    continue
                
                for time_bracket in time_brackets:
                    if x_var[(day, transport_type, current_location, dest["name"])].solution_value > 0.5:
                        next_location = dest["name"]
                        chosen_transport = transport_type
                        travel_time = transport_matrix[(current_location, next_location, time_bracket)][transport_type]["duration"]
                        price = transport_matrix[(current_location, next_location, time_bracket)][transport_type]["price"]
                        entrance_fee = dest.get("entrance_fee", 0) if dest["type"] == "attraction" else None
                        duration = (dest.get("duration", 0) // 60, dest.get("duration", 0) % 60)
                        food_price = dest.get("avg_food_price", 0) if dest["type"] == "hawker" else None
                        break
                if next_location:
                    break
            if next_location:
                break
        
        if not next_location:
            break  # No more locations for the day

        # Arrival time
        arrival_time = u_var[(day, next_location)].solution_value

        # Print movement details
        print(f"- Go to {next_location} using {chosen_transport}, travel time is {travel_time} min, price is ${price:.2f}", end="")

        if entrance_fee:
            print(f", entrance fee is ${entrance_fee:.2f}", end="")
        if duration and next_location != "DUMMY HOTEL":
            print(f", you'll be there for {duration[0]} hours {duration[1]} minutes", end="")
        if food_price:
            print(f", average food price is ${food_price:.2f}", end="")

        if next_location != "DUMMY HOTEL": print(f", finish by {int(arrival_time // 60):02d}:{int(arrival_time % 60):02d}.")

        if next_location == "DUMMY HOTEL":
            # stop because reach hotel already
            break

        current_location = next_location

    print()  # Blank line between days

Day 0:
- Start at DUMMY HOTEL by 09:00
- Go to Gardens by the Bay using transit, travel time is 50 min, price is $1.93, entrance fee is $11.00, you'll be there for 1 hours 29 minutes, finish by 12:00.
- Go to Waku Ghin using transit, travel time is 19.57 min, price is $1.19, you'll be there for 1 hours 0 minutes, average food price is $5.00, finish by 16:00.
- Go to Amoy Street Food Centre using transit, travel time is 23.4 min, price is $1.19, you'll be there for 1 hours 0 minutes, average food price is $9.00, finish by 20:25.
- Go to Sungei Road Laksa using transit, travel time is 24.28 min, price is $1.50, you'll be there for 1 hours 0 minutes, average food price is $9.00, finish by 22:00.
- Go to DUMMY HOTEL using transit, travel time is 50 min, price is $1.93, finish by 08:59.

Day 1:
- Start at DUMMY HOTEL by 09:00
- Go to Sungei Road Laksa using transit, travel time is 50 min, price is $1.93, you'll be there for 1 hours 0 minutes, average food price is $9.00, finish by 16:00.
- 